In [23]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import itertools

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", os.pardir))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import test_config, setup
from src import memory, storage
from src.utils import common, extract


test_data = test_config.get_test_data()

# Comment out if you have already initialized the database
# setup.delete_db(test_data["database"]["delete_db_script"], test_data["database"]["params"])
# setup.initialize_db(
#     test_data["database"]["create_db_script"], 
#     test_data["database"]["sql_file"], 
#     test_data["database"]["params"]
# )

responses = []
for conversation in itertools.islice(
      test_data["test"]["conversations"],
      test_data["test"]["how_many_conversations"]
):
    conversation_data = extract.extract_conversation_data(conversation)
    # storage.save_conversation_to_search_table(test_data, conversation_data)

    memory.ask_questions(test_data, conversation_data)
    # memory.generate_takeaways(test_data, conversation_data)

    # response = memory.get_response_to_conversation(test_data, conversation_data)
    #responses += [{"conversation": conversation, "response": response}]

common.save_jsonl_file(test_data["test"]["test_save_output_filepath"], responses)


An error occurred while reading /Users/nicholasbern/src/holocron/scripts/../data/templates/questions_answers/question-answer-few-shot-examples.json: [Errno 2] No such file or directory: '/Users/nicholasbern/src/holocron/scripts/../data/templates/questions_answers/question-answer-few-shot-examples.json'
An error occurred while reading /Users/nicholasbern/src/holocron/scripts/../data/templates/questions_answers/question-reframed-question-few-shot-examples.json: [Errno 2] No such file or directory: '/Users/nicholasbern/src/holocron/scripts/../data/templates/questions_answers/question-reframed-question-few-shot-examples.json'
Function _fetch_recent_messages_for_bot_room total time: 0.0056161
Function get_embeddings total time: 0.3665963
Function _fetch_similar_messages_for_bot_room total time: 0.3780304
Function get_completion total time: 2.7234469
Function _fetch_recent_messages_for_bot_room total time: 0.0058600
Function get_embeddings total time: 0.3628035
Function _fetch_similar_messag